In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.objectives import binary_crossentropy
from keras.layers import Input, Dropout, GlobalAveragePooling2D
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.optimizers import adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

Using TensorFlow backend.


In [2]:
path = "generated/"
l = os.listdir(path)
X = []

for img in l:
    image = cv2.imread(os.path.join(path,img))
    if(image.shape == (224,224,3)):
        X.append(image)


y = [0]*len(X)
 

In [3]:
path = "FakeData/"

l = os.listdir(path)

for p in l:
    
    i = os.path.join(path,p)
    
    for img in os.listdir(i):
        
        image = cv2.imread(os.path.join(i,img))
        
        if(image.shape == (224,224,3)):
            
            X.append(image)
                           
                        

y += [0]*(len(X) - len(y))

In [4]:
len(X),len(y)

(25499, 25499)

In [5]:
path = "wikiArts/"

l = os.listdir(path)

nb = 0

for p in l:
    
    z = os.path.join(path,p)
    
    for img in os.listdir(z):
        
        try:
            
            image = cv2.imread(os.path.join(z,img))
        
            if(nb < len(y) and image.shape == (224,224,3)):
            
                X.append(image)
            
                nb+=1
        
            else:
            
                break
            
        except :
                continue
    
    if(nb >= len(y)):
        break

y += [1]*nb

In [6]:
len(X),len(y)

(50998, 50998)

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
mobilenet = MobileNet(input_shape=(224,224,3), include_top=False)

input = mobilenet.layers[0].input

convolutions_output = mobilenet.layers[-1].output

out = GlobalAveragePooling2D()(convolutions_output)

fe_model = Model(inputs=input,
              outputs=out,
              name="MobileNet-224-feature-Extractor")


In [9]:
i = 0
bs = 32

transf = []
while i < X.shape[0]:
    img_batch = X[i: i+bs]
    img_batch_preproc = preprocess_input(img_batch.astype('float32'))
    transf.append(fe_model.predict(img_batch_preproc))
    i += bs

In [10]:
X = np.vstack(transf)

In [11]:
X.shape

(50998, 1024)

In [12]:
idx = np.arange(X.shape[0])

np.random.shuffle(idx)

X = X[idx]
y = y[idx]

In [13]:
data = {"paintings" : X,
        "labels" : y}

In [14]:
np.save("data.npy", data)

In [15]:
test = np.load("data.npy")[()]

In [16]:
test["paintings"].shape,test["labels"].shape

((50998, 1024), (50998,))